In [1]:
%load_ext autoreload
%autoreload 2
import sys
# map folder using sys
sys.path.append('../')
import pandas as pd
from galgoz import DATA_FOLDER
from galgoz import Galgoz
from scipy import signal
import numpy as np
from plotly import graph_objects as go
from galgoz.indicators.trend import sg
from whittaker_eilers import WhittakerSmoother
from plotly.subplots import make_subplots

gz=Galgoz()

In [2]:
filename = DATA_FOLDER / "GBP_JPY_H1.pkl"
data = pd.read_pickle(filename)
data["time"] = pd.to_datetime(data["time"])
data["time_str"] = data["time"].dt.strftime(" %-b %d, '%y %H:%M")
data[["mid_o", "mid_h", "mid_l", "mid_c"]] = data[["mid_o", "mid_h", "mid_l", "mid_c"]].astype(float)
data.set_index("time", inplace=True)

In [3]:
df_an = data.iloc[-1000:].copy()
sg_trend = sg(df_an["mid_c"])
sg_trend2 = sg(df_an["mid_c"], len=50, order=3)

In [4]:
indicators = [{"name": "Savitzy-Golay",
               "data": sg_trend,
               "mode": "lines",
               "row": 1,
               "line": {"color": "black", "width": 2}
               }]

In [5]:
fig = gz.plot_candles(df_an, indicators=indicators)
